- 修改：
    - line 9, 11: 關閉 headless mode
    - line 31: 加上 `get_attribute('outerHTML')` 以獲取 `<table>` 內容
    - line 32: 若不採用 line 31 xpath 方式，亦可採用 `soup.find` 方式獲取 `<table>` 內容
    - line 39: 加上 `index=False`，避免 csv file 輸出 index numbers，增進資料集內容品質
    - line 49: 使用 `execute_script()` 方式點擊 next page button
    - line 6, 56, 57~60: 用於評估程式從哪邊結束，有報錯需輸出 Traceback 有助於 debug

[我從瀏覽器關閉 Javascript 後發現 NBA 數據網站採用 Javascript 動態網頁，MLB 則非](https://youtu.be/Fwyes49JplM)；<br>
後來用兩組 NBA 與 MLB 數據爬蟲程式比對測試，<br>
我在 headless mode 下用等待元素出現的方式嘗試取 NBA stats `<table>`，最後 timeout 失敗，<br>
可知在 Javascript 動態網站無法用 headless mode 爬取資料；<br>
此外，若要點擊按鈕也需採用 Javascript 方法，<br>
（另外網路上有方法是採用[「等待元素出現」的方式](https://stackoverflow.com/questions/62903056/elementclickinterceptedexception-message-element-click-intercepted-element-is)，不過我尚未測試成功，）<br>
所以才會有 line 9 & 49 的修正。<br>

### References:
- [1] python selenium 瀏覽器自動化遇到Message: element click intercepted:解決方法（非延遲載入）- CSDN。取自 https://blog.csdn.net/u012874140/article/details/108869803
- [2] ElementClickInterceptedException: Message: element click intercepted Element is not clickable error clicking a radio button using Selenium and Python - Stack Overflow. 取自 https://stackoverflow.com/questions/62903056/elementclickinterceptedexception-message-element-click-intercepted-element-is
- [3] MLB.com stats crawler - GitHub. 取自 https://github.com/whyhugo/Data-Analysis-in-Sports/blob/main/mlb_stats_crawler.ipynb
- [4] NBA.com stats crawler - GitHub. 取自 https://github.com/whyhugo/LATIA112-1/blob/main/web_crawl/nba_stats_crawler.ipynb
- [5] 實測從 Google Chrome 關閉 Javascript 後，NBA.com 與 MLB.com 載入差異 - YouTube。取自 https://youtu.be/Fwyes49JplM

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import traceback

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
#chrome_options.headless = True

driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe', options=chrome_options)
driver.implicitly_wait(2)
driver.get("https://www.nba.com/stats/players/traditional")

pages_remaining = True
page_num = 1

#WebDriverWait(driver, 50).until(EC.element_to_be_clickable(('xpath', '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))).click()
while pages_remaining:
    # 使用Beautiful Soup剖析HTML網頁
    soup = BeautifulSoup(driver.page_source, "lxml")
    #print(soup.text)
    # #next > div.Layout_base6IeUC.Layout_justNav2H4H0.Layout_withSubNavByKRF > div.Layout_mainContentjXliI > div.MaxWidthContainer_mwcID5AG > section.Block_block62M07.nba-stats-content-block > div > div.Crom_basef0niE > div.Crom_container__C45Ti.crom-container > table
    #table = soup.select_one("#next > div.Layout_base6IeUC.Layout_justNav2H4H0.Layout_withSubNavByKRF > div.Layout_mainContentjXliI > div.MaxWidthContainer_mwcID5AG > section.Block_block62M07.nba-stats-content-block > div > div.Crom_basef0niE > div.Crom_container__C45Ti.crom-container > table > tbody") 
    #table = soup.select_one("body > main > div.stats-containerinner > div > div.row > div > div > nba-stat-table > div.nba-stat-table > div.nba-stat-tableoverflow > table") 
    #table = soup.select_one("body > div#next > div.Layout_base6IeUC.Layout_justNav2H4H0.Layout_withSubNavByKRF > div.Layout_mainContentjXliI > div.MaxWidthContainer_mwcID5AG > section.Block_block62M07.nba-stats-content-block > div > div.Crom_basef0niE > div.Crom_container__C45Ti.crom-container > table") 

    ### 以下則一 ###
    table = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table').get_attribute('outerHTML')
    #table = soup.find('table', class_='Crom_table__p1iZz')
    #############
    
    ##table = soup.find('table', class_='Crom_table__p1iZz')
    #print(table)
    df = pd.read_html(str(table))
    print(df[0].to_csv())
    df[0].to_csv("ALL_players_stats" + str(page_num) + ".csv", index=False) # 新增 index=False 以避免 csv file 帶有 index 值
    print("儲存頁面:", page_num)
    try:
        # 自動按下一頁按鈕
        #Copy Full XPath /html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select
        #next_link = driver.find_element(By.CSS_SELECTOR, '#next > div.Layout_base6IeUC.Layout_justNav2H4H0.Layout_withSubNavByKRF > div.Layout_mainContentjXliI > div.MaxWidthContainer_mwcID5AG > section.Block_block62M07.nba-stats-content-block > div > div.Crom_basef0niE > div.Crom_cromSettingsak6Hd > div.Pagination_contentf2at7.Crom_cromSettingTqtiq > div.Pagination_buttonsYpLUe > button:nth-child(2) > svg > polyline')
        #next_link = driver.find_element('xpath', '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]')
        #next_link = driver.find_element(By.XPATH, '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/a[2]')
        time.sleep(3)

        driver.execute_script("arguments[0].click();", driver.find_element('xpath', '/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
        
        time.sleep(5)
        if page_num < 11:
            page_num = page_num + 1
        else:
            pages_remaining = False
            print('成功結束')
    except ZeroDivisionError as e:
        pages_remaining = False
        print('失敗結束')
        traceback.print_exc()
        
driver.close()